In [2]:
# unknown words are removed (maybe affects the sequence)


In [3]:
import re
import numpy as np
import pandas as pd
import nltk
import torch
import torch.nn.functional as f
from sklearn.metrics import accuracy_score, f1_score
import time

In [4]:
glove_embedding_file = '/Users/sougata-8718/Documents/NLP/Embeddings/glove6b/glove.6B.50d.txt'

In [5]:
contents = open(glove_embedding_file, "r+").read()

In [6]:
contents = contents.split("\n")
contents = [ [x.split(" ")[0], np.array(x.split(" ")[1::])] for x in contents ][0:-1]

In [7]:
words = []
embeddings = []

for word, embedding in contents:
    words.append(word)
    embeddings.append(embedding)

In [8]:
word_idx_map = dict(zip(words, np.arange(len(words))))

In [9]:
word_embedding_map = dict(zip(words, embeddings))

In [10]:
train = open("/Users/sougata-8718/Downloads/amazon_reviews_train.csv","r+")
test = open("/Users/sougata-8718/Downloads/amazon_reviews_test.csv","r+")
train = train.read()
test = test.read()

In [11]:
def messy_text_to_df(text):
    documents = text.split("\n")
    df = pd.DataFrame()
    data = []
    labels = []
    for document in documents:
        labels.append(document.split("\t",1)[0])
        text = document.split('\t')[1]
        data.append(text)
    labels = np.array(labels)
    labels[np.where(labels=='__label__2')] = "Positive"
    labels[np.where(labels=='__label__1')] = "Negative"
    df["Data"] = data
    df["Label"] = labels
    
    return df

def remove_punctuation_and_numbers(text,replacements):
    for key,value in replacements.items():
        text = text.replace(key,value)
    text = text.translate(str.maketrans('','',';"#$%&\'()*+/<=>?@[\\]^_`{|}~0123456789')).translate(str.maketrans('!.-:,','     '))
    return text
def remove_non_words(data,replacements):
    res = data.apply(lambda x: remove_punctuation_and_numbers(x,replacements))
    return res


def remove_words_single(string,words_to_be_removed):
    words = nltk.word_tokenize(string)
    filtered_words = []
    for i in range(len(words)):
        if words[i] not in words_to_be_removed:
            filtered_words.append(words[i])
    return ' '.join(filtered_words)

def remove_words(data,words_to_be_removed):
    res = data.apply(lambda x : remove_words_single(x,words_to_be_removed))
    return res
    
    for text,label in documents:
        labels.append(document.split("\t",1)[0])
        text = document.split('\t')[1]
        for key,value in replacements.items():
            text = text.replace(key,value)
            text = text.translate(str.maketrans('','',';"#$%&\'()*+/<=>?@[\\]^_`{|}~0123456789')).translate(str.maketrans('!.-:,','     '))
        words = nltk.word_tokenize(text)
        filtered_words = []
        for i in range(len(words)):
            if words[i] not in words_to_be_removed:
                filtered_words.append(stemmer.stem(words[i]))
        
        res = ' '.join(filtered_words)
        data.append(res)
    labels = np.array(labels)
    labels[np.where(labels=='__label__2')] = "Positive"
    labels[np.where(labels=='__label__1')] = "Negative"
    return data,labels

def stem_single_string(string,nltkstemmer):
    words = nltk.word_tokenize(string)
    stemmed_list = []
    for word in words:
        stemmed_list.append(nltkstemmer.stem(word))
    return ' '.join(stemmed_list)
    

def stem(data):
    stemmer = SnowballStemmer("english")
    res = data.apply(lambda x : stem_single_string(x,stemmer))
    return res

def find_rare_words(data,max_frequency=4):
    
    vectoriser = get_vectorizer(data)
    
    
    temp = ' '.join(data)
    frequencies = (nltk.FreqDist(nltk.word_tokenize(temp)))
    
    fs = np.array(frequencies.most_common())
    fs = pd.DataFrame(fs)
    fs.columns = ["word","count"]
    fs["freq"] = fs["count"].astype(int)
    fs = fs.drop("count",axis=1)
    
    rare_words = list(fs[fs["freq"]<=max_frequency]["word"])
    
    return rare_words

def get_vectorizer(data,vectorizer="CountVectorizer"):
    
    if vectorizer == "TFIDF":
        tfidf = TfidfVectorizer()
        tfidf.fit(data)
        print("TF-IDF Vectorizer")
        return tfidf
    cv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    cv.fit(data)
    print("Count Vectorizer")
    return cv

def vectorize_data(data,vectorizer="CountVectorizer"):
    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    
    if vectorizer == "TFIDF":
        tfidf = TfidfVectorizer()
        tfidf.fit(data)
        print("TF-IDF Vectorizer")
        return tfidf.transform(data).toarray()
    cv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
    cv.fit(data)
    print("Count Vectorizer")
    return cv.transform(data).toarray()

def preprocess(data):
    data = messy_text_to_df(data)
    data["Data"] = remove_non_words(data["Data"],replacements)
    data["Data"] = data["Data"].apply(lambda x : x.lower())
    return data


In [12]:
replacements = {"can't" : 'can not',"shan't":'shall not',"won't":'will not',"'ve" : " have", "'d" : " would", "'m" : " am", "'ll" : " will", "'s" : "", "n't" : " not","'re" : "are"}
train = preprocess(train)
test = preprocess(test)

In [15]:
word_embedding_map.get("the"), np.random.randn(50)/10

(array(['0.418', '0.24968', '-0.41242', '0.1217', '0.34527', '-0.044457',
        '-0.49688', '-0.17862', '-0.00066023', '-0.6566', '0.27843',
        '-0.14767', '-0.55677', '0.14658', '-0.0095095', '0.011658',
        '0.10204', '-0.12792', '-0.8443', '-0.12181', '-0.016801',
        '-0.33279', '-0.1552', '-0.23131', '-0.19181', '-1.8823',
        '-0.76746', '0.099051', '-0.42125', '-0.19526', '4.0071',
        '-0.18594', '-0.52287', '-0.31681', '0.00059213', '0.0074449',
        '0.17778', '-0.15897', '0.012041', '-0.054223', '-0.29871',
        '-0.15749', '-0.34758', '-0.045637', '-0.44251', '0.18785',
        '0.0027849', '-0.18411', '-0.11514', '-0.78581'], dtype='<U11'),
 array([-0.0482406 ,  0.17023812, -0.00056398,  0.06081297,  0.01771576,
         0.14994409, -0.17120126, -0.12530452,  0.031613  , -0.09040955,
        -0.09735695,  0.1414364 ,  0.11935937,  0.01789004,  0.07101575,
         0.08698796,  0.00429509,  0.18495536,  0.08105742, -0.15570799,
        -0.028368

In [11]:
def sentence_to_summed_vector(sentence):
    words = nltk.word_tokenize(sentence.lower())
    word_embeddings = [embeddings[word_idx_map.get(word)] if word_idx_map.get(word)!=None else np.zeros((50,)) for word in words]
    word_embeddings = np.array(word_embeddings).astype(np.float)
    
    return np.sum(word_embeddings, axis = 0)

def sentence_to_sequence_vectors(sentence):
    words = nltk.word_tokenize(sentence.lower())
    words = [word if word_idx_map.get(word)!= None else '' for word in words]
    words = nltk.word_tokenize(' '.join(words))
    word_embeddings = [embeddings[word_idx_map.get(word)] for word in words]
    word_embeddings = torch.Tensor(np.array(word_embeddings).astype(np.float))
    #word_embeddings = word_embeddings.resize(1, word_embeddings.shape[0], word_embeddings.shape[1])
    
    return word_embeddings
    

In [93]:
X_train = torch.Tensor(train["Data"].apply(lambda x : sentence_to_summed_vector(x)))
X_test = torch.Tensor(test["Data"].apply(lambda x : sentence_to_summed_vector(x)))
Y_train = torch.Tensor([[1, 0] if x == "Positive" else [0, 1] for x in train["Label"]])
Y_test = torch.Tensor([[1, 0] if x == "Positive" else [0, 1] for x in test["Label"]])

In [133]:
class Net(torch.nn.Module):
    def __init__(self, input_shape):
        super().__init__()
        self.hidden1 = torch.nn.Linear(input_shape, 50)
        self.hidden2 = torch.nn.Linear(50, 25)
        self.output = torch.nn.Linear(25, 2)
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        pass
    
    def train(self, X_train, Y_train, X_test, Y_test, no_of_iterations):
        for i in range(no_of_iterations):
            state = self.hidden1(X_train)
            state = f.tanh(state)
            state = self.hidden2(state)
            state = f.tanh(state)
            state = self.output(state)
            state = f.softmax(state, dim = -1)

            loss = - (Y_train * torch.log(state))
            loss = torch.sum(loss)/len(X_train)
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            train_acc = accuracy_score(y_pred = torch.argmax(state.detach(), 1), y_true = torch.argmax(Y_train.detach(),1))
            train_f1 = f1_score(y_pred = torch.argmax(state.detach(), -1,), y_true = torch.argmax(Y_train.detach(),1), average = 'macro')
            #print(torch.argmax(state.detach(), 1)[0:100],torch.argmax(Y_train.detach(),1)[0:100])
            test_preds = self.forward(X_test)
            test_acc = accuracy_score(y_pred = torch.argmax(test_preds.detach(), 1), y_true = torch.argmax(Y_test.detach(),1))
            test_f1 = f1_score(y_pred = torch.argmax(test_preds.detach(), -1,), y_true = torch.argmax(Y_test.detach(),1), average = 'macro')
            print("iteration", i+1, "loss", loss.item(), "train acc/f1", round(train_acc, 3), "/",round(train_f1,3), "test acc/f1", round(test_acc, 3), "/", round(test_f1,3),"\n")
            pass
        pass
    
    def forward(self, X):
        state = self.hidden1(X)
        state = f.tanh(state)
        state = self.hidden2(state)
        state = f.tanh(state)
        state = self.output(state)
        state = f.softmax(state, dim = -1)

        return state

In [134]:
torch.manual_seed(0)

net = Net(50)
net.train(X_train, Y_train, X_test, Y_test, 500)

iteration 1 loss 0.695752739906311 train acc/f1 0.519 / 0.48 test acc/f1 0.562 / 0.562 

iteration 2 loss 0.6916160583496094 train acc/f1 0.523 / 0.519 test acc/f1 0.577 / 0.576 

iteration 3 loss 0.6871929168701172 train acc/f1 0.548 / 0.548 test acc/f1 0.578 / 0.56 

iteration 4 loss 0.6832969784736633 train acc/f1 0.558 / 0.552 test acc/f1 0.56 / 0.527 

iteration 5 loss 0.6806715130805969 train acc/f1 0.566 / 0.55 test acc/f1 0.576 / 0.55 

iteration 6 loss 0.6773461103439331 train acc/f1 0.576 / 0.564 test acc/f1 0.599 / 0.587 

iteration 7 loss 0.6734445691108704 train acc/f1 0.589 / 0.584 test acc/f1 0.623 / 0.62 

iteration 8 loss 0.669662356376648 train acc/f1 0.597 / 0.597 test acc/f1 0.621 / 0.62 

iteration 9 loss 0.6660543084144592 train acc/f1 0.607 / 0.607 test acc/f1 0.62 / 0.619 

iteration 10 loss 0.6619347929954529 train acc/f1 0.617 / 0.617 test acc/f1 0.63 / 0.627 

iteration 11 loss 0.6575955748558044 train acc/f1 0.624 / 0.624 test acc/f1 0.628 / 0.623 

iteratio

iteration 94 loss 0.46528229117393494 train acc/f1 0.781 / 0.781 test acc/f1 0.766 / 0.766 

iteration 95 loss 0.46478232741355896 train acc/f1 0.781 / 0.781 test acc/f1 0.77 / 0.77 

iteration 96 loss 0.4642857015132904 train acc/f1 0.78 / 0.78 test acc/f1 0.765 / 0.765 

iteration 97 loss 0.4637924134731293 train acc/f1 0.782 / 0.782 test acc/f1 0.774 / 0.774 

iteration 98 loss 0.4632810950279236 train acc/f1 0.782 / 0.782 test acc/f1 0.766 / 0.766 

iteration 99 loss 0.4627157747745514 train acc/f1 0.782 / 0.782 test acc/f1 0.774 / 0.774 

iteration 100 loss 0.46203678846359253 train acc/f1 0.782 / 0.782 test acc/f1 0.769 / 0.769 

iteration 101 loss 0.461482435464859 train acc/f1 0.782 / 0.782 test acc/f1 0.773 / 0.773 

iteration 102 loss 0.46087971329689026 train acc/f1 0.782 / 0.782 test acc/f1 0.773 / 0.773 

iteration 103 loss 0.46030527353286743 train acc/f1 0.783 / 0.783 test acc/f1 0.771 / 0.771 

iteration 104 loss 0.45990249514579773 train acc/f1 0.783 / 0.783 test acc/f

iteration 186 loss 0.4291337728500366 train acc/f1 0.803 / 0.802 test acc/f1 0.759 / 0.759 

iteration 187 loss 0.42888396978378296 train acc/f1 0.802 / 0.802 test acc/f1 0.767 / 0.767 

iteration 188 loss 0.42864927649497986 train acc/f1 0.803 / 0.803 test acc/f1 0.759 / 0.759 

iteration 189 loss 0.42844974994659424 train acc/f1 0.803 / 0.803 test acc/f1 0.767 / 0.767 

iteration 190 loss 0.42829838395118713 train acc/f1 0.802 / 0.802 test acc/f1 0.757 / 0.757 

iteration 191 loss 0.4281703233718872 train acc/f1 0.803 / 0.803 test acc/f1 0.769 / 0.769 

iteration 192 loss 0.42799344658851624 train acc/f1 0.803 / 0.803 test acc/f1 0.757 / 0.757 

iteration 193 loss 0.42770856618881226 train acc/f1 0.803 / 0.803 test acc/f1 0.766 / 0.766 

iteration 194 loss 0.42723360657691956 train acc/f1 0.803 / 0.803 test acc/f1 0.757 / 0.757 

iteration 195 loss 0.4266447424888611 train acc/f1 0.804 / 0.804 test acc/f1 0.761 / 0.761 

iteration 196 loss 0.42609256505966187 train acc/f1 0.803 / 0.8

iteration 277 loss 0.4022892117500305 train acc/f1 0.818 / 0.818 test acc/f1 0.758 / 0.758 

iteration 278 loss 0.402522474527359 train acc/f1 0.818 / 0.818 test acc/f1 0.76 / 0.76 

iteration 279 loss 0.4027116298675537 train acc/f1 0.818 / 0.818 test acc/f1 0.76 / 0.76 

iteration 280 loss 0.4021422863006592 train acc/f1 0.818 / 0.818 test acc/f1 0.762 / 0.762 

iteration 281 loss 0.4012068510055542 train acc/f1 0.819 / 0.819 test acc/f1 0.763 / 0.763 

iteration 282 loss 0.40096279978752136 train acc/f1 0.819 / 0.819 test acc/f1 0.761 / 0.761 

iteration 283 loss 0.4008772373199463 train acc/f1 0.82 / 0.82 test acc/f1 0.761 / 0.761 

iteration 284 loss 0.40069910883903503 train acc/f1 0.819 / 0.819 test acc/f1 0.76 / 0.76 

iteration 285 loss 0.40028896927833557 train acc/f1 0.82 / 0.82 test acc/f1 0.76 / 0.76 

iteration 286 loss 0.39978405833244324 train acc/f1 0.82 / 0.82 test acc/f1 0.76 / 0.76 

iteration 287 loss 0.39947938919067383 train acc/f1 0.82 / 0.819 test acc/f1 0.76 /

iteration 366 loss 0.37826207280158997 train acc/f1 0.834 / 0.834 test acc/f1 0.759 / 0.759 

iteration 367 loss 0.3774966299533844 train acc/f1 0.834 / 0.834 test acc/f1 0.759 / 0.759 

iteration 368 loss 0.37823933362960815 train acc/f1 0.832 / 0.832 test acc/f1 0.758 / 0.758 

iteration 369 loss 0.3790738880634308 train acc/f1 0.833 / 0.833 test acc/f1 0.755 / 0.755 

iteration 370 loss 0.3785412907600403 train acc/f1 0.831 / 0.831 test acc/f1 0.761 / 0.761 

iteration 371 loss 0.3770780563354492 train acc/f1 0.834 / 0.834 test acc/f1 0.758 / 0.758 

iteration 372 loss 0.376318097114563 train acc/f1 0.835 / 0.835 test acc/f1 0.754 / 0.754 

iteration 373 loss 0.3767380714416504 train acc/f1 0.834 / 0.834 test acc/f1 0.758 / 0.758 

iteration 374 loss 0.37717995047569275 train acc/f1 0.834 / 0.833 test acc/f1 0.757 / 0.757 

iteration 375 loss 0.37664422392845154 train acc/f1 0.833 / 0.833 test acc/f1 0.759 / 0.759 

iteration 376 loss 0.37562236189842224 train acc/f1 0.835 / 0.835 t

iteration 457 loss 0.35851672291755676 train acc/f1 0.843 / 0.843 test acc/f1 0.751 / 0.751 

iteration 458 loss 0.35839998722076416 train acc/f1 0.843 / 0.843 test acc/f1 0.754 / 0.754 

iteration 459 loss 0.3583206236362457 train acc/f1 0.843 / 0.842 test acc/f1 0.752 / 0.752 

iteration 460 loss 0.3581770062446594 train acc/f1 0.843 / 0.843 test acc/f1 0.754 / 0.754 

iteration 461 loss 0.35793837904930115 train acc/f1 0.843 / 0.843 test acc/f1 0.751 / 0.751 

iteration 462 loss 0.357650488615036 train acc/f1 0.843 / 0.843 test acc/f1 0.755 / 0.755 

iteration 463 loss 0.35738351941108704 train acc/f1 0.843 / 0.843 test acc/f1 0.755 / 0.755 

iteration 464 loss 0.3571750223636627 train acc/f1 0.844 / 0.844 test acc/f1 0.751 / 0.751 

iteration 465 loss 0.3570172190666199 train acc/f1 0.843 / 0.843 test acc/f1 0.753 / 0.753 

iteration 466 loss 0.3568745255470276 train acc/f1 0.843 / 0.843 test acc/f1 0.749 / 0.749 

iteration 467 loss 0.356717050075531 train acc/f1 0.843 / 0.843 tes

In [320]:
# loss = torch.nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)
# output
# input, target.shape, Y_test.shape
# torch.set_printoptions(3)

In [187]:
ip = ["dsfjghjbds", "the product was very nice", test.iloc[8]["Data"]]
temp = [ sentence_to_summed_vector(x) for x in ip]

net.forward(torch.Tensor(temp))

tensor([[0.5267, 0.4733],
        [0.7754, 0.2246],
        [0.7114, 0.2886]], grad_fn=<SoftmaxBackward>)

In [248]:
test.iloc[0]["Data"]

'stuning even for the non gamer  this sound track was beautiful  it paints the senery in your mind so well i would recomend it even to people who hate vid  game music  i have played the game chrono cross but out of all of the games i have ever played it has the best music  it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras  it would impress anyone who cares to listen  '

199

In [12]:
X_train = train["Data"].apply(lambda x : sentence_to_sequence_vectors(x))
X_test = test["Data"].apply(lambda x : sentence_to_sequence_vectors(x))
Y_train = torch.Tensor([[1, 0] if x == "Positive" else [0, 1] for x in train["Label"]])
Y_test = torch.Tensor([[1, 0] if x == "Positive" else [0, 1] for x in test["Label"]])

In [13]:
from keras.preprocessing.sequence import pad_sequences
max_length = max([x.shape[0] for x in X_train]) + 47

X_train = torch.Tensor(pad_sequences(X_train, maxlen=max_length, padding='pre'))
X_test = torch.Tensor(pad_sequences(X_test, maxlen=max_length, padding='pre'))

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [137]:
class RecurrentNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size = 2):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, batch_first = True)
        self.output_layer = torch.nn.Linear(hidden_size, output_size)
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        pass
    
    def train(self, X_train, Y_train, X_test, Y_test, no_of_iterations, batch_size = 100):
        
        for itr in range(no_of_iterations):
            itr_loss = 0
            
            before = time.time()
            for i in range(0, len(X_train), batch_size):
                
                X_batch = X_train[i:i+batch_size]
                Y_batch = Y_train[i:i+batch_size]
                
                lstm_outputs, (h,c) = self.lstm(X_batch)
                h = h.resize(h.shape[1], h.shape[2])
                outputs = self.output_layer(h)
                outputs = outputs.softmax(-1)
                
                batch_loss = - torch.sum(Y_batch * torch.log(outputs)) / len(X_batch) / max_length
                itr_loss+= batch_loss
                
                batch_loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                
                pass
            
            
            predictions = self.forward(X_train)
            train_acc = accuracy_score(y_pred = torch.argmax(predictions.detach(), -1), y_true = torch.argmax(Y_train, -1))
            test_preds = self.forward(X_test)
            test_acc = accuracy_score(y_pred = torch.argmax(test_preds.detach(), -1), y_true = torch.argmax(Y_test, -1))
            
            after = time.time()
            print("iteration", itr+1, "loss", round(itr_loss.item(), 6), "train acc : ", round(train_acc,4), "test acc : ", round(test_acc, 4), "time taken : ", after-before)
            pass
        pass
    
    def forward(self, X):
        lstm_outputs, (h, c) = self.lstm(X)
        h = h.resize(h.size(1), h.size(2))
        outputs = self.output_layer(h)
        outputs = outputs.softmax(-1)
        
        return outputs

In [141]:
torch.manual_seed(0)
rnet = RecurrentNet(50, 100)
rnet.train(X_train, Y_train, X_test, Y_test, 10)
# loss, outputs = rnet.single_example(X_train[1], Y_train[1])
# loss, outputs.shape
# ops = rnet.forward(X_test)
# ops.shape

/usr/local/lib/python3.7/site-packages/torch/tensor.py:339: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


KeyboardInterrupt: 

In [148]:
op, (h, c) = torch.nn.LSTM(20,50, batch_first = True, bidirectional=True)(torch.randn(75,10,20))
op.shape, h.shape, c.shape
#h.resize(h.size(1), h.size(2)).shape
torch.cat((h[0], h[1]), -1).shape, h.shape

(torch.Size([75, 100]), torch.Size([2, 75, 50]))

In [63]:
class RecurrentNetBidirectional(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size = 2):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_size, hidden_size, bidirectional = True, batch_first = True)
        self.output_layer = torch.nn.Linear(2 * hidden_size, output_size)
        self.optimizer = torch.optim.Adam(self.parameters(), lr = 0.001)
        self.loss = torch.nn.CrossEntropyLoss()
        pass
    
    def train(self, X_train, Y_train, X_test, Y_test, no_of_iterations, batch_size = 100):
        for itr in range(no_of_iterations):
            itr_loss = 0
            
            before = time.time()
            for i in range(0, X_train.shape[0], batch_size):
                
                X_batch = X_train[i:i+batch_size]
                Y_batch = Y_train[i:i+batch_size]
                
                lstm_outputs, (h,c) = self.lstm(X_batch)
                h = torch.cat((h[0],h[1]), -1)
                outputs = self.output_layer(h)
                outputs = outputs.softmax(-1)
                
                batch_loss = self.loss(input = outputs, target = Y_batch.argmax(-1))
                itr_loss+= batch_loss
                
                batch_loss.backward()
                self.optimizer.step()
                self.optimizer.zero_grad()
                
                pass
            
            predictions = self.forward(X_train)
            train_acc = accuracy_score(y_pred = torch.argmax(predictions.detach(), -1), y_true = torch.argmax(Y_train, -1))
            test_preds = self.forward(X_test)
            test_acc = accuracy_score(y_pred = torch.argmax(test_preds.detach(), -1), y_true = torch.argmax(Y_test, -1))
            
            after = time.time()
            print("iteration", itr+1, "loss", round(itr_loss.item(), 6), "train acc : ", round(train_acc,4), "test acc : ", round(test_acc, 4), "time taken : ", after-before)
            pass
        pass
    
    def forward(self, X):
        lstm_outputs, (h, c) = self.lstm(X)
        h = torch.cat((h[0], h[1]), -1)
        outputs = self.output_layer(h)
        outputs = outputs.softmax(-1)
        
        return outputs
    pass


In [64]:
torch.manual_seed(0)
rbnet = RecurrentNetBidirectional(50, 10)
rbnet.train(X_train, Y_train, X_test, Y_test, no_of_iterations = 50)


iteration 1 loss 62.562523 train acc :  0.517 test acc :  0.514 time taken :  34.75209999084473
iteration 2 loss 62.301739 train acc :  0.5511 test acc :  0.553 time taken :  35.955974102020264
iteration 3 loss 62.086433 train acc :  0.572 test acc :  0.571 time taken :  35.664610147476196


KeyboardInterrupt: 

In [80]:
ops, (h,c) = torch.nn.LSTM(50,10, batch_first = True, bidirectional = True)(torch.randn(100, 20, 50))
print(ops.shape, h.shape, c.shape)
ops, (h,c) = torch.nn.LSTM(20, 10, batch_first = True, bidirectional = True)(ops)
ops.shape, h.shape, c.shape

torch.Size([100, 20, 20]) torch.Size([2, 100, 10]) torch.Size([2, 100, 10])


(torch.Size([100, 20, 20]), torch.Size([2, 100, 10]), torch.Size([2, 100, 10]))

In [112]:
class RecurrentNetBidirectionalMultiLayer(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size = 2):
        super().__init__()
        self.lstm1 = torch.nn.LSTM(input_size, hidden_size, bidirectional = True, batch_first = True)
        self.lstm2 = torch.nn.LSTM(2 * hidden_size, hidden_size, bidirectional = True, batch_first = True)
        self.output_layer = torch.nn.Linear(2 * hidden_size, output_size)
        self.optimizer = torch.optim.SGD(self.parameters(), lr = 0.1, momentum = 0.9, weight_decay = 0.01, nesterov = True)
        self.loss = torch.nn.CrossEntropyLoss()
        pass
    
    def train(self, X_train, Y_train, X_test, Y_test, no_of_iterations, batch_size = 100):
        for itr in range(no_of_iterations):
            itr_loss = 0
            
            before = time.time()
            for i in range(0, X_train.shape[0], batch_size):
                X_batch = X_train[i:i+batch_size]
                Y_batch = Y_train[i:i+batch_size]
                
                lstm1_outputs, (h,c) = self.lstm1(X_batch)
                lstm2_outputs, (h,c) = self.lstm2(lstm1_outputs)
                h = torch.cat((h[0],h[1]), -1)
                outputs = self.output_layer(h)
                outputs = outputs.softmax(-1)
                
                batch_loss = self.loss(input = outputs, target = Y_batch.argmax(-1))
                
                itr_loss+= batch_loss / (len(X_train)/batch_size)
                
                batch_loss.backward()
                self.optimizer.step()
                
                pass
            
            predictions = self.forward(X_train)
            train_acc = accuracy_score(y_pred = torch.argmax(predictions.detach(), -1), y_true = torch.argmax(Y_train, -1))
            test_preds = self.forward(X_test)
            test_acc = accuracy_score(y_pred = torch.argmax(test_preds.detach(), -1), y_true = torch.argmax(Y_test, -1))
            
            after = time.time()
            print("iteration", itr+1, "loss", round(itr_loss.item(), 6), "train acc : ", round(train_acc,4), "test acc : ", round(test_acc, 4), "time taken : ", after-before)
            pass
        pass
    
    def forward(self, X):
        lstm1_outputs, (h, c) = self.lstm1(X)
        lstm2_outputs, (h, c) = self.lstm2(lstm1_outputs)
        h = torch.cat((h[0], h[1]), -1)
        outputs = self.output_layer(h)
        outputs = outputs.softmax(-1)
        
        return outputs
    pass


In [113]:
torch.manual_seed(0)
rbnetml = RecurrentNetBidirectionalMultiLayer(50, 10)
rbnetml.train(X_train, Y_train, X_test, Y_test, no_of_iterations = 10, batch_size = 10)

iteration 1 loss 0.703355 train acc :  0.5066 test acc :  0.5 time taken :  305.50206685066223
iteration 2 loss 0.70313 train acc :  0.5066 test acc :  0.5 time taken :  302.45164012908936
iteration 3 loss 0.70313 train acc :  0.5066 test acc :  0.5 time taken :  292.7798430919647


KeyboardInterrupt: 